In [86]:
import sys
from pathlib import Path
from typing import List, Dict

from loguru import logger
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import udf, col, when
import geopy.distance
from pyspark.sql.functions import *
from pyspark.sql import Window
import pyspark.sql.functions as F
from geopy.distance import great_circle
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit, struct
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import pow, col, log
from pyspark.sql.functions import *
from pyspark.sql import Window
import pyspark.sql.functions as F


In [71]:
CONF_LOG_PREFIX = 'CONFLOG'
FLST_LOG_PREFIX = 'FLSTLOG'
GEO_LOG_PREFIX = 'GEOLOG'
LOS_LOG_PREFIX = 'LOSLOG'
REG_LOG_PREFIX = 'REGLOG'
LOADING_PATH = '../output'
DATAFRAMES_NAMES = [CONF_LOG_PREFIX, FLST_LOG_PREFIX, GEO_LOG_PREFIX, LOS_LOG_PREFIX, REG_LOG_PREFIX]

Give access to the constants that defines

In [101]:
sys.path.append(str(Path(Path().absolute().parent, 'platform_code')))
from schemas.tables_attributes import *
from utils.config import settings

In [73]:
def load_dataframes(files_names: List[str], loading_path: str, spark: SparkSession) -> Dict[str, DataFrame]:
    """ Loads the dataframes which macht the file names passed by arguments.
    The method read from the config the path were to read the files, which
    matches the folder where the files are saved in `save_dataframes_dict()`.

    :param files_names: list of the names of the files.
    :param loading_path: path were the files are saved.
    :param spark: spark session.
    :return: dictionary with the dataframes loaded from the files, with the
     file name as key.
    """
    dataframes = dict()

    for file_name in files_names:
        file_path = Path(loading_path, f'{file_name.lower()}.parquet')
        logger.info('Loading dataframe from `{}`.', file_path)
        df = spark.read.parquet(str(file_path))
        dataframes[file_name] = df

    return dataframes

In [74]:
spark = SparkSession.builder.appName('Notebook').getOrCreate()

In [16]:
input_dataframes = load_dataframes(DATAFRAMES_NAMES, LOADING_PATH, spark)

2022-04-01 11:11:39.951 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\conflog.parquet`.
2022-04-01 11:11:40.061 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\flstlog.parquet`.
2022-04-01 11:11:40.167 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\geolog.parquet`.
2022-04-01 11:11:40.275 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\loslog.parquet`.
2022-04-01 11:11:40.360 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\reglog.parquet`.


In [76]:
@udf
def get_coordinates_distance(origin_latitude: float, origin_longitude: float,
                             destination_latitude: float, destination_longitude: float) -> float:
    """ Calculates the distance in meters between two world coordinates.

    :param origin_latitude: origin latitude point.
    :param origin_longitude: origin longitude point.
    :param destination_latitude: destination latitude point.
    :param destination_longitude: destination longitude point.
    :return: distance in meters.
    """
    origin_tuple = (origin_latitude, origin_longitude)
    destination_tuple = (destination_latitude, destination_longitude)
    # TODO: direct distance calculation (in meters) between two points, is this approach correct?
    return geopy.distance.distance(origin_tuple, destination_tuple).m

# ENV-2: Weighted average altitude
Average flight level weighed by the length flown at each flight level.

In [64]:
dataframe = input_dataframes[REG_LOG_PREFIX]

First, we check the log for a given drone in a given scenario.

In [77]:
#dataframe.where(col(SCENARIO_NAME) == '1_very_low_40_8_R2').where(col(ACID) == 'D1').orderBy(SIMULATION_TIME, ACID).show()

Create a column with the next coordinates system for the same drone

In [65]:
window = Window.partitionBy(SCENARIO_NAME, ACID).orderBy(SIMULATION_TIME)
next_step = dataframe.withColumn("NEXT_LATITUDE",lag(LATITUDE, -1).over(window)).withColumn("NEXT_LONGITUDE",lag(LONGITUDE, -1).over(window)).withColumn("NEXT_ALTITUDE",lag(ALTITUDE, -1).over(window))

In [78]:
#next_step.where(col(SCENARIO_NAME) == '1_very_low_40_8_R2').where(col(ACID) == 'D1').show(50)

In [66]:
#Remove rows with NEXT_LATITUDE and NEXT_LONGITUDE null (they are the rows of separation between scenarios)
dataframe = next_step.na.drop(subset=["NEXT_LATITUDE","NEXT_LONGITUDE"])
#Check it
dataframe.filter(col("NEXT_LATITUDE").isNull() | col("NEXT_LATITUDE").isNull()).show()

+------+--------+---------------+----+---+---+---+-------------+--------------+-------------+
|REG_ID|Scenario|Simulation_time|ACID|ALT|LAT|LON|NEXT_LATITUDE|NEXT_LONGITUDE|NEXT_ALTITUDE|
+------+--------+---------------+----+---+---+---+-------------+--------------+-------------+
+------+--------+---------------+----+---+---+---+-------------+--------------+-------------+



In [67]:
dataframe = dataframe.withColumn("SEGMENT_LENGTH",get_coordinates_distance(LATITUDE, LONGITUDE, "NEXT_LATITUDE", "NEXT_LONGITUDE")).withColumn("SEGMENT_ALTITUDE", (col(ALTITUDE)+col("NEXT_ALTITUDE"))/2)
dataframe = dataframe.withColumn("SEGMENT_WEIGHT", col("SEGMENT_ALTITUDE") * col("SEGMENT_LENGTH"))

In [79]:
#dataframe.select(REG_ID, SCENARIO_NAME, ACID, "SEGMENT_LENGTH", "SEGMENT_ALTITUDE", "SEGMENT_WEIGHT").show()

In [69]:
dataframe.groupby(SCENARIO_NAME).agg(sum(col("SEGMENT_WEIGHT")), sum(col("SEGMENT_LENGTH"))).withColumn(ENV2, col("sum(SEGMENT_WEIGHT)")/col("sum(SEGMENT_LENGTH)")).show()

+------------------+--------------------+--------------------+------------------+
|          Scenario| sum(SEGMENT_WEIGHT)| sum(SEGMENT_LENGTH)|              ENV2|
+------------------+--------------------+--------------------+------------------+
|1_very_low_40_8_W1| 1.453557703031384E9| 3.104108568569879E7|46.826896383365394|
|1_very_low_40_8_R2| 1.453557703031385E9|3.1041085685698785E7| 46.82689638336543|
|2_very_low_40_8_W1|1.4535577030313845E9| 3.104108568569877E7| 46.82689638336544|
|3_very_low_40_8_W1|1.4535577030313852E9| 3.104108568569879E7| 46.82689638336544|
|3_very_low_40_8_R2|1.4535577030313857E9|3.1041085685698777E7|46.826896383365465|
|2_very_low_40_8_R2|1.4535577030313845E9|3.1041085685698777E7| 46.82689638336543|
+------------------+--------------------+--------------------+------------------+



In [ ]:
#ENV4
dataframe4 = input_dataframes[REG_LOG_PREFIX]
dataframe4 = dataframe4.groupby(SCENARIO_NAME, ACID).agg(F.max(ALTITUDE).alias("MAX_ALTITUDE"), F.min(ALTITUDE).alias("MIN_ALTITUDE"))
dataframe4 = dataframe4.withColumn("DIFF_ALTITUDE", col("MAX_ALTITUDE") - col("MIN_ALTITUDE"))
avg_delay = dataframe4.select(mean("DIFF_ALTITUDE").alias("MEAN_DIFF_ALTITUDE"))
dataframe4 = dataframe4.join(avg_delay, how='outer')
dataframe4.show()

dataframe4 = dataframe4.withColumn(ENV4, col("DIFF_ALTITUDE")/col("MEAN_DIFF_ALTITUDE"))
dataframe4 = dataframe4.select(SCENARIO_NAME, ACID, ENV4)
dataframe4.show()
# dataframe5 = dataframe4.groupby(SCENARIO_NAME).agg(F.max('MAX_ALTITUDE').alias("MAX_ALTITUDE_SCN"), F.min('MIN_ALTITUDE').alias("MIN_ALTITUDE_SCN"))
# dataframe5.show()


In [82]:
from pyspark.sql.functions import udf
def get_coordinates_distance(origin_latitude: float, origin_longitude: float,
                             destination_latitude: float, destination_longitude: float) -> float:
    """ Calculates the distance in meters between two world coordinates.

    :param origin_latitude: origin latitude point.
    :param origin_longitude: origin longitude point.
    :param destination_latitude: destination latitude point.
    :param destination_longitude: destination longitude point.
    :return: distance in meters.
    """
    origin_tuple = (origin_latitude, origin_longitude)
    destination_tuple = (destination_latitude, destination_longitude)
    # TODO: direct distance calculation (in meters) between two points, is this approach correct?
    return geopy.distance.distance(origin_tuple, destination_tuple).m


@udf
def in_circle (x_center, y_center, x, y, radius):
    return get_coordinates_distance(x_center, y_center, x, y) <= radius

@udf("double")
def great_circle_udf(x, y):
    return great_circle(x, y).kilometers

In [105]:

print(settings.x_center)
print(settings.time_roi)
print(settings.radius_roi)

[48.15636078, 48.156360789, 48.15636071, 48.15636072, 48.15636073]
[120, 180, 240, 320, 380]
[10, 20, 30, 40, 50]


In [98]:
dataframe3.show()

+------+------------------+---------------+-----+------------------+-----------+-----------+
|REG_ID|          Scenario|Simulation_time| ACID|               ALT|        LAT|        LON|
+------+------------------+---------------+-----+------------------+-----------+-----------+
| 51200|1_very_low_40_8_R2|         2070.0| D769| 64.00800000000001|48.25804341|16.38633812|
| 51201|1_very_low_40_8_R2|         2070.0| D792|54.864000000000004|48.15463758|16.34650336|
| 51202|1_very_low_40_8_R2|         2070.0| D890|            36.576|48.16488888|16.43228973|
| 51203|1_very_low_40_8_R2|         2070.0| D894|    40.60138999848|48.20800251|16.30905314|
| 51204|1_very_low_40_8_R2|         2070.0| D895|             9.144|48.18206189|16.37362525|
| 51205|1_very_low_40_8_R2|         2070.0| D915| 64.00800000000001| 48.1626239|16.30574288|
| 51206|1_very_low_40_8_R2|         2070.0| D919|            73.152|48.24980509|16.39937127|
| 51207|1_very_low_40_8_R2|         2070.0| D923|109.72800000000001|48

In [110]:
#ENV3
dataframe3 = input_dataframes[REG_LOG_PREFIX]
udf_func = udf(great_circle_udf,DoubleType()) #Creating a 'User Defined Function' to calculate distance between two points.

#dataframe3.printSchema() 
#dataframe3.show()

for i,(x,y,t) in enumerate(zip(settings.x_center, settings.y_center, settings.time_roi)):
    point = struct(lit(x), lit(y))
    print(i,x,y,t, point)
    
    aux_dataframe = dataframe3.filter(col(SIMULATION_TIME) == t)
    aux_dataframe = aux_dataframe.withColumn("distance", great_circle_udf(point, struct(col(LATITUDE), col(LONGITUDE))))
    aux_dataframe = aux_dataframe.filter((col("distance") <= 16))
    aux_dataframe = aux_dataframe.withColumn("sound_intensity", 1/(pow((col("distance")/settings.flight_altitude.lowest), 2)))
    aux_dataframe = aux_dataframe.groupby(SCENARIO_NAME).agg(sum("sound_intensity").alias(f"ENV3_p{i}"))
    
    aux_dataframe = aux_dataframe.join(SCENARIO_NAME, )

    break


0 48.15636078 16.32453111 120 Column<'struct(48.15636078, 16.32453111)'>
+------------------+------------------+
|          Scenario|           ENV3_p0|
+------------------+------------------+
|1_very_low_40_8_W1|1410.4715514847946|
|1_very_low_40_8_R2|1410.4715514847946|
|2_very_low_40_8_W1|1410.4715514847946|
|3_very_low_40_8_W1|1410.4715514847946|
|3_very_low_40_8_R2|1410.4715514847946|
|2_very_low_40_8_R2|1410.4715514847946|
+------------------+------------------+



In [14]:


point = struct(lit(x_center), lit(y_center))
udf_func = udf(great_circle_udf,DoubleType()) #Creating a 'User Defined Function' to calculate distance between two points.
dataframe3 = dataframe3.withColumn("distance", udf_func(point, struct(col(LATITUDE), col(LONGITUDE)))).filter((col("distance") <= radius_roi) & (col(ALTITUDE)<= altitude_roi) & (col(SIMULATION_TIME) == time_roi)) #Creating column "distance" based on function 'get_distance'
#dataframe3 = dataframe3.withColumn("noise_level", 10*log(1/pow(col("distance"),2)))
dataframe3 = dataframe3.withColumn("noise_level", log10(1/pow(col("distance"),2)))
dataframe3.show()

root
 |-- REG_ID: integer (nullable = true)
 |-- Scenario: string (nullable = true)
 |-- Simulation_time: double (nullable = true)
 |-- ACID: string (nullable = true)
 |-- ALT: double (nullable = true)
 |-- LAT: double (nullable = true)
 |-- LON: double (nullable = true)

+------+------------------+---------------+-----+------------------+-----------+-----------+
|REG_ID|          Scenario|Simulation_time| ACID|               ALT|        LAT|        LON|
+------+------------------+---------------+-----+------------------+-----------+-----------+
| 76800|1_very_low_40_8_R2|         2880.0|D2320|             45.72|48.15636078|16.32453111|
| 76801|1_very_low_40_8_R2|         2880.0|D2340|128.01600000000002|48.17504097|16.36041054|
| 76802|1_very_low_40_8_R2|         2880.0|D2337|            73.152|48.23478639|16.42459103|
| 76803|1_very_low_40_8_R2|         2880.0|D2342|  136.185390000024|48.19298318|16.39227946|
| 76804|1_very_low_40_8_R2|         2880.0|D2343|          31.83255|48.21440

In [12]:
dataframe3.groupBy(SCENARIO_NAME).agg(sum("noise_level")).show()

+------------------+-------------------+
|          Scenario|   sum(noise_level)|
+------------------+-------------------+
|1_very_low_40_8_W1|-188.96845944379098|
|1_very_low_40_8_R2|-188.96845944379098|
|2_very_low_40_8_W1|-188.96845944379098|
|3_very_low_40_8_W1|-188.96845944379098|
|3_very_low_40_8_R2|-188.96845944379098|
|2_very_low_40_8_R2|-188.96845944379098|
+------------------+-------------------+



In [ ]:
dataframe3.show()

In [ ]:



dataframe = input_dataframes[REG_LOG_PREFIX]

point = struct(lit(settings.x_center), lit(settings.y_center))

# TODO: ? Define formula for the sound depending on the distance to the point.
# TODO: ? How many points and how to define.
return dataframe.filter(col(SIMULATION_TIME) == settings.time_roi) \
    .withColumn("distance", great_circle_udf(point, struct(col(LATITUDE), col(LONGITUDE)))) \
    .filter((col("distance") <= settings.radius_roi) & (col(ALTITUDE) <= settings.altitude_roi)) \
    .withColumn("noise_level", log10(1 / pow(col("distance"), 2))) \
    .groupBy(SCENARIO_NAME).agg(sum("noise_level").alias(ENV3))